<a href="https://colab.research.google.com/github/IsaacHgo/Generacion-de-nombres-usando-un-modelo-LSTM/blob/main/Nombres_de_Hormigas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import nltk
from nltk.corpus import names
import random
import string
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
#Para que usted no tenga que cargar ninguna base de datos cvs lo que hare es
#extraer mis datos desde una libraría de nombres
Todos = list(set(names.words('male.txt') + names.words('female.txt')))
#Aqui solo hago la base de datos sucia de nombres
Todos[:20]

['Jose',
 'Gaylene',
 'Mona',
 'Rae',
 'Mattheus',
 'Lucia',
 'Cappella',
 'Karee',
 'Dinny',
 'Halette',
 'Tasia',
 'Matilda',
 'Cora',
 'Monroe',
 'Arlene',
 'Aube',
 'Demetra',
 'Ursula',
 'Poul',
 'Cecilia']

In [4]:
#Los uno en una sola linea de texto para que pueda adaptarse a lo que quiero hacer
unanomas = "\n".join(Todos)
unanomas

"Jose\nGaylene\nMona\nRae\nMattheus\nLucia\nCappella\nKaree\nDinny\nHalette\nTasia\nMatilda\nCora\nMonroe\nArlene\nAube\nDemetra\nUrsula\nPoul\nCecilia\nTansy\nErtha\nToinette\nJean-Francois\nAvis\nGavriel\nSondra\nSusi\nMarney\nKeslie\nSkye\nHammad\nAndrei\nDanella\nFernandina\nJessamyn\nJessika\nJaquenetta\nSelinda\nOfelia\nTruman\nJeralee\nRustie\nWendel\nKaril\nClaudelle\nLilyan\nKathlene\nDarelle\nRonalda\nBurton\nKay\nTheda\nDianemarie\nGenny\nJethro\nCorie\nRobbie\nMeghann\nMildred\nFarand\nSturgis\nFlorance\nOdelle\nFelicio\nZonnya\nSusie\nJanka\nJourdan\nZacherie\nReeva\nWoody\nClayborn\nNorton\nSkelly\nHamlen\nEmmalee\nGretal\nDebbi\nDanila\nErnie\nUlrica\nCortese\nRichy\nLaurice\nPrunella\nReine\nDaune\nCharlotta\nLibbey\nGlorianna\nLilli\nMarja\nPenrod\nAinslie\nHyacintha\nBarret\nElwyn\nTremain\nMadelena\nShelagh\nGerrard\nAl\nJeniece\nTamarah\nHonoria\nEvelyn\nShirley\nBerkie\nSara\nJeffie\nZorah\nLeeAnn\nMareah\nHector\nHerve\nBetta\nAnnabella\nSuellen\nYetta\nLorne\nAxe

In [5]:
caracteres = sorted(list(set(unanomas)))
Char_Inex = {ch: i for i, ch in enumerate(caracteres)}
Index_Char = {i: ch for i, ch in enumerate(caracteres)}
#Dos diccionarios para las transformaciones futuras, pura desencriptacion pues
Nalpha = len(caracteres) #Número total de nuestro alfabeto
Unaseq = 6  #Una secuencia
Char_Inex

{'\n': 0,
 ' ': 1,
 "'": 2,
 '-': 3,
 'A': 4,
 'B': 5,
 'C': 6,
 'D': 7,
 'E': 8,
 'F': 9,
 'G': 10,
 'H': 11,
 'I': 12,
 'J': 13,
 'K': 14,
 'L': 15,
 'M': 16,
 'N': 17,
 'O': 18,
 'P': 19,
 'Q': 20,
 'R': 21,
 'S': 22,
 'T': 23,
 'U': 24,
 'V': 25,
 'W': 26,
 'X': 27,
 'Y': 28,
 'Z': 29,
 'a': 30,
 'b': 31,
 'c': 32,
 'd': 33,
 'e': 34,
 'f': 35,
 'g': 36,
 'h': 37,
 'i': 38,
 'j': 39,
 'k': 40,
 'l': 41,
 'm': 42,
 'n': 43,
 'o': 44,
 'p': 45,
 'q': 46,
 'r': 47,
 's': 48,
 't': 49,
 'u': 50,
 'v': 51,
 'w': 52,
 'x': 53,
 'y': 54,
 'z': 55}

In [6]:
#Dos matrices que ocuparemos para tranformar los datos de tal manera que tengamos
#Variables inependientes y respuesta, es decir
#Que para una secuencia directa de caracteres la variable de respuesta sea
#el siguiente caracter, es decir f(jua)=n, f(isaa)=c,f(migu)=e
#Esto para que el modelo haga algo como:

#f(P)=a, f(Pa)=b, f(Pab)=l, f(Pabl)=o, f(Pablo)=\n
#R=Pablo
X = []
y = []

for i in range(len(unanomas) - Unaseq):
    x_seq = unanomas[i:i+Unaseq]#Se leé un tamaño de "Unaseq" osea lo que creo yo que es el tamaño de un nombre promedio
    y_des = unanomas[i+Unaseq]#Esta es la letra instante despues de seq, es decir, la variable de respuesta f(x_seq)=y_des

    X.append([Char_Inex[c] for c in x_seq]) #Se rellena la matriz de vectores que en lugar de lo leido en letras coloca su index
    y.append(Char_Inex[y_des]) #Lo mismo, pero con la variable de respuesta

# Convertir a one-hot
X_onehot = np.zeros((len(X), Unaseq, Nalpha), dtype=np.bool_) #Se crea una matriz de vectores de Ceros de tamaño de lo leido
y_onehot = np.zeros((len(X), Nalpha), dtype=np.bool_) ##Se crea una matriz de vectores de Ceros de tamaño de lo leido, pero con la de respuesta

for i, seq in enumerate(X): #Este for es para crear vectores de ceros y un 1 tal que cada uno sea asignado para cada indice y lo mismo para la varable de respuesta
#Así se tienen 6 Vectores que dan de respuesta a un vector
    for j, idx in enumerate(seq):
        X_onehot[i, j, idx] = 1
        y_onehot[i, y[i]] = 1

In [7]:
#Este solo es el modelo hecho en la cual no hay mucho que decir
#se explica medio solo, y el 128 es porque lo vi comun xd, no entendi
#muy bien ese tema
model = Sequential([
    LSTM(128, input_shape=(Unaseq, Nalpha), return_sequences=True),
    Dropout(0.2),
    LSTM(128),
    Dense(Nalpha, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')
#Usamos Adam porque aprende rápido y se adapta solo. Combina ideas que ayudan
#a que la red aprenda mejor sin tanto ajuste. Funciona bien en muchos casos.
#Es como un optimizador "inteligente y práctico"




# ============================
# 🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥aqui se pueden cambiar las epocas 👇👇👇 (No sabia que se podian poner emojis en phyton, chicle pega ajahaa)
# ============================
model.fit(X_onehot, y_onehot, batch_size=18, epochs=10)
#Se ajusta el modelo, lo haré solo de 10 epocas para que pueda hacerlo de manera rápido

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2978/2978 ━━━━━━━━━━━━━━━━━━━━ 44s 14ms/step - loss: 2.7883


In [8]:
#Esto en si solo es bueno, una forma de leer el modelo
def sample(preds):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds + 1e-8)#Las prediciones las convierte en probabilidades con la ficion probit o similar, no recuerdo askajskjas
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)#El chiste es que tenemos la probabilidad
    probas = np.random.multinomial(1, preds, 1)#Mediante las probabilidades de la ultima red con la multibinomial se eliga de maner "aleatoria" una de las letras
    #del alfabeto
    return np.argmax(probas)#Se devuelve el vector de letra predicho por las probabilidades del modelo y la aleatoridad de la miltibinomial





def generate_name(semilla, Tamamo):
    PalabraNew = semilla#Esto es para la actualizcion de la palabra generada, volver a leer al final, pues es la letra inicial plus lo predicho
    for _ in range(Tamamo):
        x_pred = np.zeros((1, Unaseq, Nalpha))#La prediccion se llena de ceros
        for t, char in enumerate(semilla[-Unaseq:]):#se lee las ultimas letras de la semilla, esto para predecir con respecto a lo original, se supone que gracias
        #Al modelo tambien recordara patrones de nombres
            x_pred[0, t, Char_Inex[char]] = 1 #Se hace el vector predico de ceros y unos

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, )
        next_char = Index_Char[next_index]#En sumen los vectores de ceros y unos de traducen en letras

        PalabraNew += next_char#Se actuzliza la nombre pedicha
        semilla += next_char#Se actualiza la semilla para que siga creciendo

    return PalabraNew#Se regresa el nombre

# Probar generación
#random.choice(string.ascii_uppercase es una letra mayuscula al azar :p
print()
for nombre in generate_name(random.choice(string.ascii_uppercase), Tamamo=1000).split('\n'):
    print(f"Hay una hormiga que se llama se llama: {nombre} 🐜")#Esto es una tonteria mia que queria poner


Hay una hormiga que se llama se llama: Sa 🐜
Hay una hormiga que se llama se llama: -rcin 🐜
Hay una hormiga que se llama se llama: Kaneto 🐜
Hay una hormiga que se llama se llama: Jama 🐜
Hay una hormiga que se llama se llama: Shopi 🐜
Hay una hormiga que se llama se llama: Malth 🐜
Hay una hormiga que se llama se llama: Harta 🐜
Hay una hormiga que se llama se llama: Jonelonno 🐜
Hay una hormiga que se llama se llama: Rokoneo 🐜
Hay una hormiga que se llama se llama: Sarula 🐜
Hay una hormiga que se llama se llama: Epenelne 🐜
Hay una hormiga que se llama se llama: Erira 🐜
Hay una hormiga que se llama se llama: Frerdy 🐜
Hay una hormiga que se llama se llama: Yelalini 🐜
Hay una hormiga que se llama se llama: Runellda 🐜
Hay una hormiga que se llama se llama: Shennth 🐜
Hay una hormiga que se llama se llama: Wreudta 🐜
Hay una hormiga que se llama se llama: Dielbard 🐜
Hay una hormiga que se llama se llama: Suutha 🐜
Hay una hormiga que se llama se llama: Elov 🐜
Hay una hormiga que se llama se llama: